In [1]:
import numpy as np
import sympy

def generate_lwe_data(n=32, num_samples=100000):
    # Step 1: Fix a variable q as a prime number between 2^15 and 2^30
    q = sympy.randprime(2**15, 2**30)

    # Step 2: Fix 'n' as the dimensions and a binary secret 's' as {0,1}^n
    s = np.random.randint(2, size=n)

    lwe_data = []
    for _ in range(num_samples):
        # Step 3: Sample 'a' uniformly from Z^n
        a = np.random.randint(0, q, size=(n,))
        # Step 4: Compute 'b' such that b = a.s mod q to get (A, b) pairs
        b = (a.dot(s) + np.random.randint(0, 2)) % q
        lwe_data.append((a,b))

    return q, s, lwe_data

In [2]:
q, s, lwe_data = generate_lwe_data()

# Display the generated data
print("Prime q:", q)
print("Binary secret s:", s)
print("Sample LWE data pairs:")
for i in range(5):  # Displaying the first 5 pairs for illustration
    print(f"A: {lwe_data[i][0]}, b: {lwe_data[i][1]}")


Prime q: 585606479
Binary secret s: [1 1 1 0 1 0 1 1 1 0 1 1 1 0 1 1 0 0 1 1 0 0 0 0 0 1 0 1 1 1 0 0]
Sample LWE data pairs:
A: [ 13042277 444761893 508170355 492064689   1305041   2088726 399027951
  54786609 582364421  82394293 244151755 258220703  64140245 458527322
 399220203 255644149 366533740 121140517 432151340 342409117  67994919
 270340270 556801219 507265651 269394167  69633460 282069124 563534705
 174238317  46216398 477678549 320157990], b: 558051643
A: [424004595 256693704 220302781 569073958  49453279 218098611 339145157
  13099360 389626915 346005492 212070092 427298454  11549799 195497396
 443101803 510965920   6422471 320670092 248856370  81815088 526249902
  90265857 387863178  52783266 383996843 398158507 461705716 186582232
  24814086  63840784   6394381 360192030], b: 6411630
A: [445875931 102687284 327314279  38029166 485502092 455407841 134822852
 200244314 217504106  60521950 499759887 560239735 547209970 442803617
 496279889  12663528 583810068 548954901 50859

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Function to create the generator model
def build_generator(latent_dim, output_dim):
    model = keras.Sequential()
    model.add(layers.Dense(256, input_dim=latent_dim, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dense(output_dim, activation='linear'))
    return model

# Function to create the discriminator model
def build_discriminator(input_dim):
    model = keras.Sequential()
    model.add(layers.Dense(512, input_dim=input_dim, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='linear'))
    return model
'''
def build_discriminator(input_dim):
    model = keras.Sequential()
    model.add(layers.Dense(128, input_dim=input_dim, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    return model

def wasserstein_loss(y_true, y_pred):
    return tf.reduce_mean(y_true * y_pred)
'''

# Function to compile the discriminator with Wasserstein loss
def compile_discriminator(discriminator):
    discriminator.compile(optimizer=keras.optimizers.RMSprop(lr=0.00005), loss='binary_crossentropy')
    return discriminator

# Function to build the Wasserstein GAN model
def build_wasserstein_gan(generator, discriminator):
    discriminator.trainable = False
    model = keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(optimizer=keras.optimizers.RMSprop(lr=0.00005), loss='binary_crossentropy')
    return model

In [ ]:
# Function to train Wasserstein GAN
def train_wasserstein_gan(generator, discriminator, gan, lwe_data, latent_dim, epochs=500, batch_size=64):
    for epoch in range(epochs):
        # Train discriminator
        idx = np.random.randint(0, len(lwe_data), size=batch_size)
        real_samples = np.array([lwe_data[i][0] for i in idx])
        noise = np.random.randn(batch_size, latent_dim)
        fake_samples = generator.predict(noise)

        real_labels = np.ones((batch_size, 1))
        fake_labels = -np.ones((batch_size, 1))

        d_loss_real = discriminator.train_on_batch(real_samples, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_samples, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator
        noise = np.random.randn(batch_size, latent_dim)
        valid_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_labels)

        # Print progress
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, D Loss: {d_loss}, G Loss: {g_loss}")

In [ ]:
if __name__ == "__main__":
    latent_dim = 100
    n = 64
    output_dim = 64

    # Generate LWE data
    q, _, lwe_data = generate_lwe_data(n, num_samples=100000)

    # Build and compile models
    generator = build_generator(latent_dim, output_dim)
    discriminator = build_discriminator(input_dim=n)
    discriminator = compile_discriminator(discriminator)
    gan = build_wasserstein_gan(generator, discriminator)

    # Train Wasserstein GAN
    train_wasserstein_gan(generator, discriminator, gan, lwe_data, latent_dim)

In [ ]:
generated_samples = generator.predict(np.random.rand(10, latent_dim))
print("Generated Samples:")
print(generated_samples)